In [ ]:
import os
import sys
sys.path.append(os.path.abspath(".."))
print(sys.path)
from sheet_manager import SpreadsheetManager
from dotenv import load_dotenv
import requests
from utilities import current_sg_time
from datetime import datetime
import numpy as np

load_dotenv('../.env.dev')

In [ ]:
import subprocess

try:
    # Run the system command and capture the output
    result = subprocess.run(['systemctl', 'status', 'postgresql'], 
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE,
                            text=True)
    print(result.stdout)
    # Check if PostgreSQL is active
    if 'inactive (dead)' in result.stdout:
        print("PostgreSQL service is inactive.")
    # else:
    #     result = subprocess.run(['systemctl', 'stop', 'postgresql'],
    #                         stdout=subprocess.PIPE, 
    #                         stderr=subprocess.PIPE,
    #                         text=True)
    #     print("PostgreSQL service stopped.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import psycopg2
from contextlib import closing

def execute_cmd(cmd):
    try:
        with closing(psycopg2.connect(dbname='chatbot', user=os.environ.get('SQL_USER'), password=os.environ.get('PGPASSWORD'), host='localhost', port=os.environ.get('SQL_PORT'))) as conn:
            with conn.cursor() as cur:
                cur.execute(cmd)
                rows = cur.fetchall()
                col_names = [desc[0] for desc in cur.description]
                print("Connection to the database was successful")
                # Process your rows here
                return rows, col_names
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
users_rows, users_col_names = execute_cmd("SELECT * FROM users;")

In [ ]:
col_order = ['name', 'alias', 'number', 'dept', 'reporting_officer_name', 'is_global_admin', 'is_dept_admin']

In [ ]:
import pandas as pd

db_users = pd.DataFrame(users_rows, columns=users_col_names)
db_users = db_users[col_order]
db_users.sort_values(by="name", inplace=True)
db_users

In [ ]:
USERS_TABLE_URL = f"https://graph.microsoft.com/v1.0/drives/{os.environ.get('DRIVE_ID')}/items/{os.environ.get('USERS_FILE_ID')}/workbook/worksheets/MainTable/tables/MainTable/rows"

with open("../token.txt", 'r') as f:
    token = f.read()

header = {
    'Authorization': token,
    'Content-Type': 'application/json'
}

response = requests.get(url=USERS_TABLE_URL, headers=header)

data = [tuple(info) for object_info in response.json()['value'] for info in object_info['values']]
az_users = pd.DataFrame(data=data, columns=["name", "alias", "number", "dept", "reporting_officer_name", "access"])
az_users['is_global_admin'] = (az_users['access'] == 'GLOBAL')
az_users['is_dept_admin'] = (az_users['access'] == 'DEPT')
az_users.drop(columns=["access"])
az_users.sort_values(by="name", inplace=True)
az_users = az_users[col_order]
az_users = az_users.replace({np.nan: None, "": None})
az_users

In [ ]:
exact_match = az_users.equals(db_users)

if exact_match:
    print("no changes")

else:
    print("changes made")

In [ ]:
merged_data = pd.merge(az_users, db_users, how="outer", indicator=True)
old_data_names = merged_data[merged_data['_merge'] == 'right_only']['name']
new_data_names = merged_data[merged_data['_merge'] == 'left_only']['name']

old_data = db_users.loc[(db_users['name'].isin(old_data_names) | db_users['reporting_officer_name'].isin(old_data_names))]
old_data = old_data.replace({np.nan: None})

new_data = az_users.loc[(az_users['name'].isin(new_data_names) | az_users['reporting_officer_name'].isin(new_data_names))]
new_data = new_data.replace({np.nan: None})

old_users_tuples = [name for name in old_data['name']]
new_users_tuples = [tuple(new_user) for new_user in new_data.values]

print(f"Old users: {old_data['name'].values}")
print(f"New users: {new_data['name'].values}")

display(old_data)
display(new_data)

In [ ]:
leave_rows, leave_col_names = execute_cmd("SELECT date, users.name, users.dept, job_leave.leave_type FROM leave_records JOIN job ON job.job_no = leave_records.job_no JOIN job_user ON job.job_no = job_user.job_no JOIN users ON job_user.name = users.name JOIN job_leave ON job.job_no = job_leave.job_no WHERE leave_records.is_cancelled = False;")

In [ ]:
mc_records_df = pd.DataFrame(leave_rows, columns=leave_col_names)
# mc_records_df = mc_records_df[mc_records_df['is_cancelled'] != True]
mc_records_df

In [ ]:
dept_order = ('Corporate', 'ICT', 'AP', 'Voc Ed', 'Lower Pri', 'Upper Pri', 'Secondary', 'High School', 'Relief')

In [ ]:
groups = mc_records_df.groupby("dept")

dept_dfs = []

for (dept, data) in groups:
    print(dept)
    print(data)
    dept_admins, dept_admins_col_names = execute_cmd(f"SELECT * FROM users WHERE dept = '{dept}' AND is_dept_admin = True;")
    dept_admins_df = pd.DataFrame(dept_admins, columns=dept_admins_col_names)
    dept_dfs.append(dept_admins_df)
    

In [ ]:
for df in dept_dfs:
    display(df)